In [20]:
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [21]:
import requests

import pandas as pd
import numpy as np
import copy
import json

from ast import literal_eval

import torch
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from transformers import pipeline
from transformers import GPT2TokenizerFast
from PIL import Image

import pickle

In [22]:
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict

import sklearn.datasets as datasets
import sklearn.manifold as manifold

In [23]:
import openai
import os
import sys
import dotenv
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity
import tiktoken

api_path="C:/Users/2hy03/gpt_test.env"
load_dotenv(api_path) # .env 파일에서 환경 변수 로드
api_key = os.getenv("OPENAI_API_KEY") # API 키 가져오기
openai.api_key=os.getenv("OPENAI_API_KEY") 

In [24]:
cur_os = sys.platform

In [6]:
model_path = f"D:/github" if cur_os.startswith('win') else None

In [7]:
minwon_metadata= pd.read_csv('minwon.csv', sep=",", dtype=str)
print(minwon_metadata.shape)
minwon_metadata.head()

(521, 4)


,title,content,answer,depart
0,주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과
2,9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과
3,46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 내용: 요즘 가게 망하게 만드는 일들만 일어나네요..요(영종도 호텔)요즘 전...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과


# HuggingFace embedding

- DistilBERT(Distilled BERT) 모델

    -BERT 모델의 경량화 버전
    
    -WordPiece 토크나이저 사용
    
    -BERT 모델보다 빠른 추론 속도와 적은 메모리 사용량
    
    -대용량 데이터가 없는 상황에서도 잘 작동함
    
 
- GPT-2이용해보기

In [190]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v2')

In [191]:
minwon_metadata['hf_embeddings'] = minwon_metadata['content'].apply(lambda x : model.encode(x))
minwon_metadata.shape

KeyboardInterrupt: 

In [ ]:
minwon_metadata.head()

In [123]:
minwon_metadata.to_csv('minwon_metadata_hfem.csv',encoding='utf-8-sig',index=False)

-----

# OpenAI Embedding

In [9]:
minwon_metadata= pd.read_csv('minwon.csv', sep=",", dtype=str)
df=minwon_metadata
df.head()

,title,content,answer,depart
0,주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과
2,9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과
3,46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 내용: 요즘 가게 망하게 만드는 일들만 일어나네요..요(영종도 호텔)요즘 전...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과


In [10]:
#token수 확인
## gpt-3.5-turbo 모델에 맞는 token encoding model은 cl100k_base이다.
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

#contents 컬럼의 토큰 수를 계산하여 새로운 컬럼으로 만들어 붙임
df['tokens'] = df.content.apply(lambda x: num_tokens_from_string(x, 'cl100k_base'))
#del df['Unnamed: 0']
df = df.set_index(keys='title')
df.head()

,content,answer,depart,tokens
title,,,,
주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과,85
뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과,1097
9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과,555
46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,236
살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 내용: 요즘 가게 망하게 만드는 일들만 일어나네요..요(영종도 호텔)요즘 전...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과,1490


In [11]:
df['depart'].value_counts()

도시개발국 교통운수과       71
도시개발국 건축과         68
도시개발국 건설과         62
국제도시건설국 기반시설과     29
국제도시건설국 도시공원과     23
도시개발국 환경보호과       18
국제도시건설국 교통과       18
기획감사실             18
관광진흥실             17
국제도시행정국 친환경조성과    17
보건소 보건행정과         16
주민생활지원국 청소과       15
홍보체육진흥실           14
도시관리국 교통행정과       13
용유출장소             13
주민생활지원국 자원순환과     13
보건소 건강증진과         13
국제도시국 교통지적과       13
국제도시건설국 농수산과      13
국제도시국 기반시설과       12
도시재생국 도시개발과       12
운서동               11
주민생활지원국 환경관리과     11
주민생활지원국 위생과       11
Name: depart, dtype: int64

In [15]:
df.tokens.mean()

397.90140845070425

In [16]:
#토큰수1200개 이상 삭제
df = df[df['tokens'] <= 1200]
df

,content,answer,depart,tokens
title,,,,
주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과,85
뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과,1097
9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과,555
46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,236
신흥교통 46번버스회사 고발합니다,민원 내용:신흥교통 46번버스회사 고발46번버스이용중 불편사항 해당교통회사 전화문의...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,178
...,...,...,...,...
을왕리해수욕장 식당호객행위 및 주차장 장악,민원 내용:을왕리해수욕장 식당호객행위 및 주차장 장악결혼기념일을 맞아 주말 3월2일...,"1. 먼저, 모처럼의 나들이에 을왕리해수욕장 내 호객행위로 인해 심려를 끼쳐 드린 ...",용유출장소,300
영마루공원 풋살경기장 이용료 받아 소음방지시설 갖춰 주세요.,민원 내용:영마루공원 풋살경기장 이용료 받아 소음방지시설 갖춰 주세요.항상 구민 복...,새올 전자민원창구 민원상담 접수에 따라 붙임과 같이 답변하고자 합니다.붙 임 1. ...,홍보체육진흥실,506
관내 보건소 명단 요청합니다.,민원 내용:관내 보건소 명단 요청.치과기공소 현황 문의 드립니다.의료관련 기관을 검...,"○ 저희 보건행정 업무에 관심 가져 주신 점 감사드리며, 귀하께서 제기한 민원에 대...",보건소 건강증진과,253


In [17]:
trans = df[df['depart'] == '도시개발국 교통운수과']
trans

,content,answer,depart,tokens
title,,,,
주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과,85
9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과,555
46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,236
신흥교통 46번버스회사 고발합니다,민원 내용:신흥교통 46번버스회사 고발46번버스이용중 불편사항 해당교통회사 전화문의...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과,178
인천항 연안여객 터미널 도서 주민 주차하기 너무 불편합니다.,민원 내용:인천항 연안여객 터미널 도서 주민 주차하기 불편.옹진군 도서주민이인천항...,1. 평소 구정발전을 위해 관심 가져주시는 귀하께 깊은 감사를 드립니다.2. 귀하께...,도시개발국 교통운수과,454
...,...,...,...,...
신흥동2가 불법주차신고,민원 내용:신흥동2가 불법주차신고신흥도2가 헬로우 스프엔커피점앞 횡단보도및자전거도로...,1. 구정 발전을 위해 관심을 가져 주신데 대해 감사드립니다. 2. 【생활불...,도시개발국 교통운수과,73
인천기독병원 응급실앞 불법주차신고,민원 내용:인천기독병원 응급실앞 불법주차신고이 스타렉스 불법주차때매 119구급차량이...,어이가없네요 응급실앞 불법주차한차를 봐주다니 응급환자오면 불법주차때매 죽을수도있다는...,도시개발국 교통운수과,119
율목동 서해대로483번길 여전히 인도 주차개선 안되고있음,민원 내용:율목동 서해대로483번길 여전히 인도 주차개선 안되고있음주차 개선했주다면...,1. 구정 발전을 위해 관심을 가져 주신데 대해 감사드립니다. 2. 귀하께서...,도시개발국 교통운수과,147


In [199]:
sort_df=df.sort_values(by='tokens', ascending=False)
sort_df

,content,answer,depart,tokens
title,,,,
월미도 어느 식당의 바가지 요금..,민원 내용:월미도 어느 식당의 바가지 요금..오늘 휴일이라 아이 데리고 월미도에갔다...,새올민원상담란으로 접수된 민원(접수번호: 6091)에 대하여 다음과 같이 답변하고자...,주민생활지원국 위생과,1178
인천유림노르웨이숲에듀오션 공사 시작된건가요?,민원 내용:인천유림노르웨이숲에듀오션 공사 시작된건가요?1. 20191127에 신청한...,1. 귀하의 무궁한 발전을 기원합니다.2. 귀하께서 접수하신 「인천유림노르웨이숲에듀...,도시개발국 건축과,1177
건설현장으로인해 집이무너지게 생겼어요,민원 내용:건설현장으로인해 집이무너지게 생겼어요건설과에 문의드린바휴대폰번호도알려드렸...,1. 평소 구정발전에 많은 관심을 가져 주심에 감사드립니다.2. 우리 구 신흥동1가...,도시개발국 건축과,1160
복지정책을역행하는공무원!,민원 내용:복지정책을역행하는공무원!지금부터적는게 제가격은 황당한일입니다...오늘 어...,운서동 사회복지 담당 정종민입니다. 우선 민원처리 과정에서 불편을 끼쳐드려 죄송합니...,운서동,1145
인하대병원 옆의 정석빌딩 사설주차장의 개선이 필요합니다.,민원 내용:인하대병원 옆의 정석빌딩 사설주차장의 개선이 필요.업무보느라 고생 많으십...,1. 평소 구정발전을 위해 관심 가져주시는 귀하께 깊은 감사를 드립니다.2. 귀하께...,도시개발국 교통운수과,1101
...,...,...,...,...
아파트관리소장의 관리업무 지도 감독 요청,민원 내용:아파트관리소장의 관리업무 지도 감독 요청첨부화일을 참조 바랍니다.,1. 귀하께서 국민신문고로 접수하신 민원사항에 대하여 공동주택팀 관리업무를 담당하고...,도시개발국 건축과,46
학교앞 담배꽁초,민원 내용:학교앞 담배꽁초학교앞 담배꽁초 문제 해결해주세요,담배 없는 우리지역 만들기에 관심을 가져 주셔서 감사합니다귀하께서 제보하신 학교앞 ...,보건소 건강증진과,42
화물운송회사 횡포에 대한 민원청구,민원 내용:화물운송회사 횡포에 대한 민원청구,항상 구정업무에 관심을 가져주셔서 대단히 감사합니다.1. 지입차주께서 2007년부터...,도시개발국 교통운수과,31


In [200]:
sort_df.iloc[10]['tokens']

1069

In [18]:
trans['tokens'].sum()

26898

In [19]:
#자연어->embedding벡터
##embedding 모델 : text-embedding-ada-00

COMPLETIONS_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"

def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
        # index는 그대로 유지하고 content에 있는 내용만 embedding vector로 바꾼 새로운 데이터 프레임 생성
    }

In [104]:
document_embeddings2 = compute_doc_embeddings(trans)

RateLimitError: Rate limit reached for default-global-with-image-limits in organization org-cnv7grROqpjupKgCshyrU34X on requests per min. Limit: 60 / min. Please try again in 1s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

In [45]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [ ]:
# embedding example
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

---

# ChapGPT이용

- 3개의 chatgpt이용
      - 1개는 민원을 요약하는 것
      - 1개는 비슷한 유형의 민원을 찾는 것-> 답변생성과 부서추천
           -embedding을 기반으로 유사도를 계산해 비슷한 민원의 정보를 가져옴
           -민원 답변과 부서도 함께 loading
      - 1개는 비슷한 유형의 민원과의 비슷한 패턴으로 답변 생성

## 1.민원 요약

### 필요한 prompt 형태 설정

- 요약의 경우, 
       민원내용: "새로운 민원 input"
       상황: "###"
       요약: "###"

In [35]:
def print_summary(msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']

In [36]:
"""
messages = [
    {"role": "system", "content": "You are a bot that summarizes the complaints in Jung-gu, Incheon. summarize the situation and content of the new complaint"},
    {"role": "user", "content": "If I give you the contents of the complaint, present the situation and summary of the complaint."}
]


print_msg(messages)
"""

'\nmessages = [\n    {"role": "system", "content": "You are a bot that summarizes the complaints in Jung-gu, Incheon. summarize the situation and content of the new complaint"},\n    {"role": "user", "content": "If I give you the contents of the complaint, present the situation and summary of the complaint."}\n]\n\n\nprint_msg(messages)\n'

In [42]:
query = input("새로운 민원: ")

새로운 민원: 중구에 실미유원지 해수욕장 폐쇄라는 안내를 봤는데요저녁시간에 텐트치고 고기구어먹는거는 감독안하시나요그런게 다 코로나19 방역에 문제가 될 수 있는거 아닐까요관리하시는 분들 퇴근하시면 아예 문을 잠그시던가 해야지 그게  제대로 이루어지고 있는 폐쇄조치인지 의문스럽고 잠깐이라도 방역수칙 지키며 찾아가는 사람으로서 불안감을 떨칠 수가 없습니다 안그래도 잠깐 갔을 때 저녁시간엔 어떻게 되는 것일까 궁금하고 걱정되긴 했었는데 텐트치고 상피는 사람 눈으로도 보고sns를 보니 누가봐도  폐쇄한 해수욕장 이라고 생각할 수 없겠어요정확한 방역수칙을 지키는 사람으로서 아주 불안한 한 마음에 그냥 넘어갈 수 없어 신고합니다 중구에 실미유원지 해수욕장 폐쇄라는 안내를 봤는데요저녁시간에 텐트치고 고기구어먹는거는 감독안하시나요그런게 다 코로나19 방역에 문제가 될 수 있는거 아닐까요관리하시는 분들 퇴근하시면 아예 문을 잠그시던가 해야지 그게  제대로 이루어지고 있는 폐쇄조치인지 의문스럽고 잠깐이라도 방역수칙 지키며 찾아가는 사람으로서 불안감을 떨칠 수가 없습니다 안그래도 잠깐 갔을 때 저녁시간엔 어떻게 되는 것일까 궁금하고 걱정되긴 했었는데 텐트치고 상피는 사람 눈으로도 보고sns를 보니 누가봐도  폐쇄한 해수욕장 이라고 생각할 수 없겠어요정확한 방역수칙을 지키는 사람으로서 아주 불안한 한 마음에 그냥 넘어갈 수 없어 신고합니다


In [41]:
messages = [
    {"role": "system", "content": "You are a bot that summarizes the complaints in Jung-gu, Incheon. summarize the situation and content of the new complaint. 모든 대답은 한글로."},
    {"role": "user", "content" :"새로운 민원:{} \n상황: \n요약:\n상황과 요약을 작성해".format(query)}
]


print_summary(messages)

'중구에 위치한 실미유원지 해수욕장의 폐쇄 조치가 제대로 이루어지고 있는지 의문을 제기하는 새로운 민원이 제출되었다. 텐트를 설치하고 고기를 구워 먹는 등 방역수칙을 지키지 않는 행위가 빈번히 일어나고 있어 걱정과 불안감을 느끼는 시민들의 목소리가 높아지고 있다는 것이다.'

## 2. 민원답변 생성

### 임베딩

In [7]:
#SentenceTransformer 모델 이용
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v2')

In [8]:
#임베딩벡터 df 불러오기
minwon_embedding = pd.read_csv('minwon_metadata_hfem.csv', header=0,index_col=None)
minwon_embedding.head()

,title,content,depart,hf_embeddings
0,주차장이 너무 어두워요,민원 제목:주차장이 너무 어두워요\n 민원 내용:중구 전동 25328 번지옆 전동...,도시재생국 교통운수과,[-5.35281181e-01 2.37762824e-01 -2.85491236e-...
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 제목:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?\n 민...,국제도시건설국 농수산과,[-6.08929753e-01 1.08436748e-01 -1.65535067e-...
2,9100번 광역버스 배차간격 준수 요청 건,민원 제목:9100번 광역버스 배차간격 준수 요청 건\n 민원 내용:집안 사정으로 ...,도시재생국 교통운수과,[-4.80863005e-01 1.58284277e-01 1.40753791e-...
3,46번버스 신흥교통 정상이아닌 버스업체,민원 제목:46번버스 신흥교통 정상이아닌 버스업체\n 민원 내용:46번버스 신흥교통...,도시재생국 교통운수과,[-3.78309190e-01 5.16959392e-02 1.14686184e-...
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 제목:살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종...,국제도시건설국 도시공원과,[-4.00836587e-01 1.59474164e-01 5.24328053e-...


In [ ]:
def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    return {
        idx: get_embedding(r.combined) for idx, r in df.iterrows()
        # index는 그대로 유지하고 combined에 있는 내용만 embedding vector로 바꾼 새로운 데이터 프레임 생성
    }

In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
data = pd.read_csv('minwon.csv', sep=",")

# 모델 불러오기
model = SentenceTransformer('paraphrase-distilroberta-base-v2')

# content 컬럼 벡터화
data['content_embedding'] = list(model.encode(data['content']))


In [10]:
data.to_csv('minwon_hfem.csv',encoding='utf-8-sig',index=False)

In [7]:
# 새로운 민원 입력받기
new_complaint = input("새로운 민원: ")

# 새로운 민원 벡터화
new_complaint_vec = model.encode([new_complaint])

# 코사인 유사도 계산
similarity_scores = cosine_similarity(new_complaint_vec, list(data['content_embedding']))[0]

# 유사도가 높은 민원 3개 불러오기
top_indices = similarity_scores.argsort()[::-1][:5]
top_complaints = data.iloc[top_indices]

새로운 민원: 쓰레기 무단 투기 검사


In [8]:
# 결과 출력하기
print("새로운 민원과 유사한 민원 Top 3:")
for i, row in top_complaints.iterrows():
    print("새로운 민원 :{}".format(new_complaint))
    print(f"{i+1}. {row['content']} (유사도: {similarity_scores[i]:.2f})")
    print("-------------------------------------------------------------")


새로운 민원과 유사한 민원 Top 3:
새로운 민원 :쓰레기 무단 투기 검사
98. 민원 제목:신흥동 해광사 옆 계단에 가로등 설치 요구
 민원 내용:해광사 옆 공사관계로 통행로인 해광사 옆 계단이 너무 어둡습니다 안전한 통행을 위해서 임시가로등이라도 설치해 주기를 바랍니다
 민원 답변
: 보안등 2개가 설치되어 매우 밝아졌습니다. 만족합니다. (유사도: 0.84)
-------------------------------------------------------------
새로운 민원 :쓰레기 무단 투기 검사
708. 민원 제목:공항신도시 쓰레기 투기 관련 조치요청
 민원 내용:영종도 공항신도시 거주민입니다공항신도시 내의 쓰레기 투기와 관련한 민원입니다쓰레기를 버리는 장소가 아닌 곳에 많은 쓰레기가 지속적으로 투기되고 있습니다이로 인해 통행에 불편을 주어 시민안전을 저해하고 있고 위생 및 미관상 좋지 않고 더럽고 관리되지 않는 곳이라는 이미지를 주고 있습니다인터넷 다음wwwdaumnet 지도 서비스 로드뷰를 통해서 같은 장소를 확인해 보니 역시 쓰레기가 투기되어 있어 상시적으로 투기되고 관리되지 않는 것으로 보입니다쓰레기가 투기되는 장소를 다음과 같이 알려드리오니 무단으로 쓰레기가 투기되지 않도록 조치하여 주시기 바랍니다1 영종도 공항신도시 소방서 맞은편 주차장 옆 인도2 영종도 공항신도시 외국인특화거리 한복판 세부장소  사진 참조 첨부파일  공항신도시 외국인 특화거리 한복판_20171008jpg공항신도시 소방서 맞은편 주차장 옆 인도_20171008jpg다음 로드뷰 사진jpg
 민원 답변
: 우리 洞에 관심을 갖고 참여해 주신데 대해 감사드리며, 그로 인해 불편을 드린 점 양해바랍니다.귀하께서 문의하신 불법 쓰레기 투기는 현장을 확인하여 처리하였습니다.추후 청소 현장순찰로 보다 깨끗한 거리가 될 수 있도록 청소환경에 적극 노력하겠습니다. 기타 문의사항이 있으시면 아래 연락처로 문의하여 주시기 바랍니다. 감사합니다* 기타 문의사항 : 운서동 (760-6257

In [ ]:
#BERT기반->bert-base-uncased
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# 문장 입력받기
sentence1 = "The quick brown fox jumps over the lazy dog"
sentence2 = "A quick brown fox jumps over a lazy dog"

# 토큰화 및 인코딩
inputs1 = tokenizer.encode_plus(sentence1, add_special_tokens=True, return_tensors='pt')
inputs2 = tokenizer.encode_plus(sentence2, add_special_tokens=True, return_tensors='pt')

# 문장 임베딩 벡터 생성
with torch.no_grad():
    embeddings1 = model(inputs1['input_ids'], token_type_ids=None)[0].mean(dim=1)
    embeddings2 = model(inputs2['input_ids'], token_type_ids=None)[0].mean(dim=1)

# 코사인 유사도 계산
similarity = F.cosine_similarity(embeddings1, embeddings2)
print(similarity.item())


In [1]:
data

NameError: name 'data' is not defined

In [69]:
# 새로운 민원과 유사한 민원 찾기
def find_similar_complaints(new_complaint, data, model, num_similar_complaints=3):
    # 새로운 민원과 데이터 프레임의 민원들을 벡터화
    new_complaint_vec = vectorize_text(new_complaint, model)
    data_vec = vectorize_df(data, model)
    #data_vec=minwon_embedding['hf_embeddings'].tolist()
    

    # 유사도 계산하기
    similarity_scores = cosine_similarity([new_complaint_vec], data_vec)[0]

    # 유사도가 높은 민원 추출하기
    top_indices = similarity_scores.argsort()[::-1][:num_similar_complaints]
    top_complaints = minwon_embedding.iloc[top_indices]

    # 결과 출력하기
    print("새로운 민원과 유사한 민원 Top 3:")
    for i, row in top_complaints.iterrows():
        print(f"{i+1}. {row['content']} (유사도: {similarity_scores[i]:.2f})")
    
    return top_complaints


In [71]:
# 새로운 민원과 유사한 민원 찾기
new_complaint = "도로에 구멍이 많아서 차량 운행이 어려워요"
similar_complaints = find_similar_complaints(new_complaint, minwon_embedding, model)

NameError: name 'vectorize_text' is not defined

In [61]:
data_vec=minwon_embedding['hf_embeddings'].tolist()

In [62]:
#새로운 민원 입력
new_complaint = input("새로운 민원을 입력하세요: ")
# 새로운 민원 정제하기
new_complaint = new_complaint.strip()
new_complaint_vec = model.encode([new_complaint])

새로운 민원을 입력하세요: 실미도 짜증


In [63]:
similarity_scores = cosine_similarity(new_complaint_vec, data_vec)[0]

ValueError: could not convert string to float: '[-5.35281181e-01  2.37762824e-01 -2.85491236e-02  2.46142522e-01\n  2.37398908e-01  3.60307127e-01  3.82632613e-01 -8.88311677e-03\n -1.15752764e-01  1.42021462e-01  3.80301833e-01 -1.53642029e-01\n -1.23459928e-01  2.07639411e-01 -3.63507748e-01 -3.42081755e-01\n  4.74310040e-01  7.41067231e-02 -3.63065362e-01  4.54661936e-01\n  8.28482583e-02 -1.14243247e-01 -8.45964968e-01 -3.16479832e-01\n -5.74448287e-01 -1.91054698e-02 -7.73969531e-01 -5.79280496e-01\n -5.14554121e-02 -2.47274458e-01 -4.69121903e-01  5.40114120e-02\n -1.02262706e-01 -1.91872701e-01 -9.15929899e-02 -5.27599990e-01\n -2.17337832e-01 -3.25925350e-02  5.09218335e-01  8.43315274e-02\n  1.05539672e-01 -1.58860505e-01  1.44038737e-01  9.47724730e-02\n -2.55467564e-01  1.05679221e-01 -4.47466433e-01  1.09237373e-01\n -4.57591772e-01 -5.54057896e-01 -7.63673559e-02 -3.64116281e-01\n  3.24843377e-02  7.68033639e-02  1.14039890e-01 -3.54275554e-02\n  1.69224858e-01 -3.71398300

In [52]:
# 유사도 계산하기
similarity_scores = cosine_similarity(new_complaint_vec, data_vec)[0]

# 유사도가 높은 민원 추출하기
top_indices = similarity_scores.argsort()[::-1][:3]
top_complaints = data.iloc[top_indices]

# 결과 출력하기
print("새로운 민원과 유사한 민원 Top 3:")
for i, row in top_complaints.iterrows():
    print(f"{i+1}. {row['content']} (유사도: {similarity_scores[i]:.2f})")

ValueError: could not convert string to float: '[-5.35281181e-01  2.37762824e-01 -2.85491236e-02  2.46142522e-01\n  2.37398908e-01  3.60307127e-01  3.82632613e-01 -8.88311677e-03\n -1.15752764e-01  1.42021462e-01  3.80301833e-01 -1.53642029e-01\n -1.23459928e-01  2.07639411e-01 -3.63507748e-01 -3.42081755e-01\n  4.74310040e-01  7.41067231e-02 -3.63065362e-01  4.54661936e-01\n  8.28482583e-02 -1.14243247e-01 -8.45964968e-01 -3.16479832e-01\n -5.74448287e-01 -1.91054698e-02 -7.73969531e-01 -5.79280496e-01\n -5.14554121e-02 -2.47274458e-01 -4.69121903e-01  5.40114120e-02\n -1.02262706e-01 -1.91872701e-01 -9.15929899e-02 -5.27599990e-01\n -2.17337832e-01 -3.25925350e-02  5.09218335e-01  8.43315274e-02\n  1.05539672e-01 -1.58860505e-01  1.44038737e-01  9.47724730e-02\n -2.55467564e-01  1.05679221e-01 -4.47466433e-01  1.09237373e-01\n -4.57591772e-01 -5.54057896e-01 -7.63673559e-02 -3.64116281e-01\n  3.24843377e-02  7.68033639e-02  1.14039890e-01 -3.54275554e-02\n  1.69224858e-01 -3.71398300

In [50]:
#비슷한 임베딩값 찾기
def vector_similarity(x: list[float], y: list[float]) -> float:
    
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:

    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [ ]:
order_document_sections_by_query_similarity("주차장", minwon_embedding)[:5]